# Data Club 18/07/2019

## Introduction to handling and manipulating data in Python

In [1]:
# Load the required packages and get access to CATMAID

import pymaid
import pandas as pd
pymaid.set_pbars(jupyter=False, leave=False)
pymaid.set_loggers('WARNING')
# Set catmaid login instance
rm = pymaid.CatmaidInstance('https://neuropil.janelia.org/tracing/fafb/v14',
                           'fly',
                           'superfly',
                           '5db25732b0560af54d41a67e49777e5233f1f229')

## Getting Neurons

We have annotated 10 excitatory mPNs ('Rclub_1807_Ex') and 10 Inhibitory mPNs ('Rclub_1807_In') in CATMAID. We can then use the `pymaid.get_neurons()` function to load these neurons as two neuron lists, one of the excitatory neurons, and one for the inhibitory. We save these neuron lists as `In` and `Ex` . Information on neuron lists as a data structure can be found here: 

https://pymaid.readthedocs.io/en/latest/source/generated/pymaid.CatmaidNeuronList.html#pymaid.CatmaidNeuronList

In brief, a neuron list is a special instance of a pandas DataFrame - which we will be using in a second, see here: 

https://www.tutorialspoint.com/python_pandas/python_pandas_dataframe

Using typing name of the variable eg: `Ex` we can view the entire table, or alternatively `Ex.head()` will show the top 5 rows of the neuron list.

In [2]:
In = pymaid.get_neurons('annotation:Rclub_1807_In')
Ex = pymaid.get_neurons('annotation:Rclub_1807_Ex')
In.head()

,neuron_name,skeleton_id,n_nodes,n_connectors,n_branch_nodes,n_end_nodes,open_ends,cable_length,review_status,soma
0,mPN mlALT VP1+2 LTS 0.99 3813443 RJVR,3813442,4008,315,155,160,114,1071.808706,NA,True
1,mPN mlALT VM5d+4 LTS 0.98 3813435 RJVR,3813434,5366,192,151,158,92,1436.165635,NA,True
2,mPN mlALT VP1+VA1v+DL2v+DA1+33 LTS 0.40 276618...,2766186,21328,910,924,942,611,5311.400367,NA,True
3,mPN mlALT VP1+5 LTS 0.98 3813484 ASB,3813483,3118,130,109,111,64,805.984388,NA,True
4,mPN mlALT VP1+5 LTS 0.99 3813425 RJVR,3813424,3247,160,113,117,58,821.812585,NA,True


## Shifting data about

The two neuron lists have a lot of information in which we don't need yet, so here it would be good to merge them both into a single table that has all the information we need - including some that is not in the neuron list. For all 20 neurons (both lists), we want a table of Neuron name, Skid, Tract, Excitatory or Inhibitory, and Type. 

We can take the neuron name data, and the skid data from the neuron list. Although long winded, it is easier(read:lazier) to create two data frames then merge them together in a second, although takes a couple more lines of code. To create a data frame in python we use the `pandas` toolbox, which we imported as pd (check the second line of code...), so to call a function from within pandas we can use `pd.<function>`, in this case `DataFrame`. Info on the DataFrame function can be found here:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

In this example we are passing a `dictionary` object to the function, which we created and called `data`. A dictionary is simply any set of objects, each with a unique identifying name, or `Key`. In our case, the key is the column name, and the data we want to put into the column is the object. In python `{}` creates a dictionary, and they are formated as { `Key_1` : `Object_1` , `Key_2` : `Object_2` } . 

So how do we get a single column from a neuron list (and a data frame, it works the same way). For our data frame `Ex` and column `neuron_name` one way to do it is simply `Ex.neuron_name` . Once we creat our dictionary, and call it data, we can pass this to the `pd.DataFrame` function. Alternatively, we could have created the dictionary within the DataFrame fuction: `pd.DataFrame({'Neuron_name':Ex.neuron_name,'Skid':Ex.skeleton_id})`

We called our data frame `Excitatory` . As with neuron lists, we can view the first 5 rows using the `.head()` function

In [3]:
data = {'Neuron_name':Ex.neuron_name,'Skid':Ex.skeleton_id}
Excitatory = pd.DataFrame(data)
Excitatory.head()

,Neuron_name,Skid
0,Multiglomerular PN mALT 57217 GA,57216
1,Multiglomerular PN mALT DA1 Type 2 R 57180 LK,57179
2,Multiglomerular PN mALT 57127 LK,57126
3,Multiglomerular mALT PN 57040 IJA,57039
4,Multiglomerular mALT PN 57077 ML,57076


# Adding columns

We next wanted to add a `Tract` , `Ex/In` , and `Type` column. Adding a column to a data frame (df) can be done using `df[<desired column name>] = <whatever data>` . In this example, as within the excitatory data frame, the data in each column is the same so we can just pass a string with the label we want, and pandas will repeat it as a column. Generally, the column you are adding can be anything as long as it has the same number of rows as the data frame you are adding it to.

In [4]:
Excitatory['Tract'] = 'mALT'
Excitatory['Ex/In'] = 'Excitatory'
Excitatory['Type']  = '1'

We then repeat everyting for the Inhibitory data, and merge together the two data frames into one. We use the pandas `concat` function to merge the data frames. we give it both data frames, grouped together using the `[]`. after the comer in the function input you will also see  `ignore_index = True`. This tells pandas to, rather than keep the existing inputs (0:9 for both data frames), but rather replace the index with `0 : n-1`. If we don't do this the index would be 0:9 repeated twice.

In [5]:
Inhibitory = pd.DataFrame({'Neuron_name':In.neuron_name,'Skid':In.skeleton_id})
Inhibitory['Tract'] = 'mlALT'
Inhibitory['Ex/In'] = 'Inhibitory'
Inhibitory['Type']  = '2'
# merge the two data frames together
data = pd.concat([Excitatory,Inhibitory],ignore_index = True)
data

,Neuron_name,Skid,Tract,Ex/In,Type
0,Multiglomerular PN mALT 57217 GA,57216,mALT,Excitatory,1
1,Multiglomerular PN mALT DA1 Type 2 R 57180 LK,57179,mALT,Excitatory,1
2,Multiglomerular PN mALT 57127 LK,57126,mALT,Excitatory,1
3,Multiglomerular mALT PN 57040 IJA,57039,mALT,Excitatory,1
4,Multiglomerular mALT PN 57077 ML,57076,mALT,Excitatory,1
5,Multiglomerular PN mALT 57205 IJA,57204,mALT,Excitatory,1
6,Multiglomerular PN mALT 57209 IJA,57208,mALT,Excitatory,1
7,Multiglomerular mALT PN 32794 Flywalkies JMR,32793,mALT,Excitatory,1
8,Multiglomerular mALT PN DA1 Type 1b R 57020 GA,57019,mALT,Excitatory,1
9,Multiglomerular PN unknown 37213 JMR,37212,mALT,Excitatory,1


# Indexing, and editing specific values

The very first column - 0:19, is the index. It maybe easier to change this to something more imediately useful, like the skid. Setting a column to be the index is simple in pandas Data Frames, and works the same way for neuron lists:

In [6]:
data = data.set_index('Skid',drop=True)
data.head()

,Neuron_name,Tract,Ex/In,Type
Skid,,,,
57216,Multiglomerular PN mALT 57217 GA,mALT,Excitatory,1
57179,Multiglomerular PN mALT DA1 Type 2 R 57180 LK,mALT,Excitatory,1
57126,Multiglomerular PN mALT 57127 LK,mALT,Excitatory,1
57039,Multiglomerular mALT PN 57040 IJA,mALT,Excitatory,1
57076,Multiglomerular mALT PN 57077 ML,mALT,Excitatory,1


There are three further things we may want to do:

    Subset the dataset based on the index
    Subset the dataset based on values in one of the columns
    Update specific values in a dataframe
    
There are a million and 1 ways to index in pandas - and just as many online tutorials, for example: 

https://brohrer.github.io/dataframe_indexing.html

It is worth reading up further...

For simplicity here though, say we just want to pull the data for neuron `3813483`. As the skids are set as the index, this can be done using the index values and the `.loc` function, which takes index values in square brackets:

In [7]:
data.loc['3813483']

Neuron_name    mPN mlALT VP1+5 LTS 0.98 3813484 ASB
Tract                                         mlALT
Ex/In                                    Inhibitory
Type                                              2
Name: 3813483, dtype: object

As we only pulled one row of the data frame, it returns a pandas `series` rather than a dataframe, this isn't important for now, it just look different.

If we wanted to pull multiple rows, we need to pass the `.loc` function a list of index values (skids) we are interested in. In python, lists are created using `[]`, much like creating dictionaries with `{}`. As before, we can either creare a list and have it as a variable we pass to the .loc function, or just create a list within the function call (note the double square brackets, and that as we pull more than one row, it returns a dataframe, not a series - PYMAID WORKS THE SAME WAY):

In [8]:
data.loc[['3813483','57076']]

,Neuron_name,Tract,Ex/In,Type
Skid,,,,
3813483,mPN mlALT VP1+5 LTS 0.98 3813484 ASB,mlALT,Inhibitory,2
57076,Multiglomerular mALT PN 57077 ML,mALT,Excitatory,1


We've already seen how to get a single column from a pandas data frame, but we can also use `.loc` to get a column, multiple columns, or a subset of just 2 specific rows and the data in 2 specific columns, for example. The input to `.loc` follows a `[row , column]` format, but the column section can be left blank. If you want to use .loc to extract a full column you need to explicitly tell pandas to get all rows though, so you need to use `:` so - `[:,<column of interest>]`. As before, you can pass a list of columns. If you are only interested in specific rows you can replace `;` with the index or list of indicies you are interested in:

In [9]:
data.loc[:,'Neuron_name'].head()

Skid
57216                 Multiglomerular PN mALT 57217 GA
57179    Multiglomerular PN mALT DA1 Type 2 R 57180 LK
57126                 Multiglomerular PN mALT 57127 LK
57039                Multiglomerular mALT PN 57040 IJA
57076                 Multiglomerular mALT PN 57077 ML
Name: Neuron_name, dtype: object

In [10]:
data.loc[:,['Neuron_name','Tract']].head()

,Neuron_name,Tract
Skid,,
57216,Multiglomerular PN mALT 57217 GA,mALT
57179,Multiglomerular PN mALT DA1 Type 2 R 57180 LK,mALT
57126,Multiglomerular PN mALT 57127 LK,mALT
57039,Multiglomerular mALT PN 57040 IJA,mALT
57076,Multiglomerular mALT PN 57077 ML,mALT


In [11]:
data.loc[['3813483','57076'],['Neuron_name','Tract']]

,Neuron_name,Tract
Skid,,
3813483,mPN mlALT VP1+5 LTS 0.98 3813484 ASB,mlALT
57076,Multiglomerular mALT PN 57077 ML,mALT


It is also usefull to be able to extract data based on some specific quality of the data itself, for example if we don't know the skids which are in group 1, we can't search by the index! We can easily pull data based on criteria however - for example subset a dataframe for just the values in the type column that are equal to 1:

In [12]:
data[data.Type == '1']

,Neuron_name,Tract,Ex/In,Type
Skid,,,,
57216,Multiglomerular PN mALT 57217 GA,mALT,Excitatory,1
57179,Multiglomerular PN mALT DA1 Type 2 R 57180 LK,mALT,Excitatory,1
57126,Multiglomerular PN mALT 57127 LK,mALT,Excitatory,1
57039,Multiglomerular mALT PN 57040 IJA,mALT,Excitatory,1
57076,Multiglomerular mALT PN 57077 ML,mALT,Excitatory,1
57204,Multiglomerular PN mALT 57205 IJA,mALT,Excitatory,1
57208,Multiglomerular PN mALT 57209 IJA,mALT,Excitatory,1
32793,Multiglomerular mALT PN 32794 Flywalkies JMR,mALT,Excitatory,1
57019,Multiglomerular mALT PN DA1 Type 1b R 57020 GA,mALT,Excitatory,1


We can use nearly any logical expression or boolian array to subset our data frame. In this example we use something called a `list comprehension`, which embeds a for loop into a single statement and returns a list, to determine which of the neurons have `Nibelung` within their name, and use the output boolian array to subset data...

In [13]:
data[['Nibelung' in i for i in data.Neuron_name]]

,Neuron_name,Tract,Ex/In,Type
Skid,,,,
2766186,mPN mlALT VP1+VA1v+DL2v+DA1+33 LTS 0.40 276618...,mlALT,Inhibitory,2
3003322,mPN mlALT VP1+DP1m+33 LTS 0.45 3003323 Twin of...,mlALT,Inhibitory,2


We may want to update specific values within an array - so lets change the type of the Nibelung neurons to 3. we do this using the `.at` function, which gets subsets exactly the same way as `.loc`. remeber the `[row, column]` format. We can specify the row using the list comprehension from above, then we set the columns as 'Type', and assign a new value like so:

In [14]:
data.at[['Nibelung' in i for i in data.Neuron_name],'Type'] = 3
data

,Neuron_name,Tract,Ex/In,Type
Skid,,,,
57216,Multiglomerular PN mALT 57217 GA,mALT,Excitatory,1
57179,Multiglomerular PN mALT DA1 Type 2 R 57180 LK,mALT,Excitatory,1
57126,Multiglomerular PN mALT 57127 LK,mALT,Excitatory,1
57039,Multiglomerular mALT PN 57040 IJA,mALT,Excitatory,1
57076,Multiglomerular mALT PN 57077 ML,mALT,Excitatory,1
57204,Multiglomerular PN mALT 57205 IJA,mALT,Excitatory,1
57208,Multiglomerular PN mALT 57209 IJA,mALT,Excitatory,1
32793,Multiglomerular mALT PN 32794 Flywalkies JMR,mALT,Excitatory,1
57019,Multiglomerular mALT PN DA1 Type 1b R 57020 GA,mALT,Excitatory,1


## Finally...

Robs task: Generate a connectivity matrix showing numbers of synapses between the 20 mPNs and all their downstream partners

In [19]:
N_all = pymaid.CatmaidNeuronList([In,Ex])
test = pymaid.get_partners(N_all.skeleton_id)
test.head()

,neuron_name,skeleton_id,num_nodes,relation,3813442,3813434,2766186,3813483,3813424,3903440,...,57179,57126,57039,57076,57204,57208,32793,57019,37212,total
0,VUM multiglomerular PN lALT 1340264 lALT Eponi...,12070989,13605,upstream,0,0,27,0,18,0,...,0,0,0,0,0,0,0,0,0,76.0
1,VUM multiglomerular PN lALT 4954526 SD ZM,4954525,12043,upstream,0,0,22,0,19,0,...,0,0,0,0,0,0,0,0,0,68.0
2,mPN mlALT VP1+DP1m+33 LTS 0.45 3003323 Twin of...,3003322,25739,upstream,0,0,54,0,0,0,...,0,0,0,0,0,0,0,0,0,54.0
3,Left DA1 Downstream t2 tract-like 1794927 RJVR...,1794926,21028,upstream,0,0,3,0,0,0,...,0,0,0,0,5,5,33,0,1,49.0
4,mPN mlALT VP1+VA1v+DL2v+DA1+33 LTS 0.40 276618...,2766186,21328,upstream,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47.0


In [20]:
# Subset to only downstream neurons
test = test[test.relation == 'downstream']
# Remove the neuron name, number of nodes, and relation columns
test = test.drop(['neuron_name','num_nodes','relation','total'],axis = 1)
# Set the index of the dataframe to the skids
test = test.set_index('skeleton_id')
# transpose the data so our mPNs are the rows
test = test.T

test

skeleton_id,2766186,2115466,3003322,2153398,554327,4905365,5851983,1311287,1967862,2794070,...,3788293,12002,4779469,23569,57543,3136002,3503698,7021255,7021482,1198633
3813442,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3813434,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2766186,0,0,47,4,25,24,0,24,11,10,...,0,0,0,0,0,0,0,0,0,0
3813483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3813424,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3903440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3813335,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,1
3003322,54,0,0,0,18,16,0,13,17,17,...,0,0,0,0,0,0,0,0,0,0
3813403,0,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3842493,0,5,0,0,0,0,0,0,5,1,...,0,0,0,0,0,0,0,0,0,0
